In [ ]:
import wfdb
from wfdb import processing
import math
import pywt
import pandas as pd
import numpy as np
from numpy import load
import scipy.io as sio
import numpy.matlib 
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import signal
from scipy.signal import butter, lfilter
from scipy import stats
from scipy.integrate import simps
from scipy.interpolate import interp1d

from os import listdir
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from pandas import ExcelWriter
from openpyxl import load_workbook

from numpy import * #isnan
import json
import csv
import seaborn as sn

In [ ]:
def filtra(x):
    
    """Implementacao dos filtros passa-baixas e passa-altas de Pan-Tompkins.

    Args:
        x: numpy array. Sinal de ECG ao qual deve ser aplicado o filtro passa-faixa de Pan-Tompkins.

    Return:
        y2: numpy array.

    """
    
    # Filtro passa-baixas
    b1 = np.array([1, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 1])
    a1 = np.array([1, -2, 1])
    y1 = signal.lfilter(b1, a1, x)
    y1 = y1/36 #corrige o ganho
    y1 = y1[5:] #corrige o delay

    # Filtro passa-altas
    b2 = np.array([-1/32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/32])
    a2 = np.array([1, -1])
    y2 = signal.lfilter(b2, a2, y1)
    y2 = y2[16:] #corrige o delay
    
    return y2

In [ ]:
def filtra_baixa(x):
        
    """Implementacao do filtro passa-baixas de Pan-Tompkins.

    Args:
        x: numpy array. Sinal de ECG ao qual deve ser aplicado o filtro passa-baixas de Pan-Tompkins.

    Return:
        y1: numpy array. 

    """
    
    # Filtro passa-baixas
    b1 = np.array([1, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 1])
    a1 = np.array([1, -2, 1])
    y1 = signal.lfilter(b1, a1, x)
    y1 = y1/36 #corrige o ganho
    y1 = y1[5:] #corrige o delay

    return y1

In [ ]:
def detectaRR(x, fs):
            
    """ Calculo dos intervalos RR para estimativa do periodo medio do batimento
    sem o uso dos limiares dinamicos de Pan-Tompkins.

    Args:
        x: numpy.ndarray. Sinal de ECG a partir do qual se deseja estimar os intervalos RR.
        fs: int. Frequencia de amostragem do sinal.

    Return:
        periodo: numpy.float64. Estimativa do periodo de um batimento do sinal x
            em numero de amostras. 
        
    """
    
    y2 = filtra(x)

    # Filtro derivativo
    a3 = 0.1*np.array([2, 1, 0, -1, -2])
    y3 = np.convolve(a3, y2)
    y3 = y3/0.1
    y3 = y3[2:]

    # Elevando ao quadrado
    y4 = y3**2
    y = y4
    
    # Algoritmo 1: Estimativa do periodo medio
    max_peak = np.max(y)
    limite = max_peak*0.1
    RR = []
    subida = 0
    ultimo = 0

    for i in range(np.size(y)):
        if (y[i] > limite):
            if (subida == 0):
                ti = i - ultimo
                RR.append(ti)
                ultimo = i;
            subida = 30;
        else:
            if (subida > 0):
                subida = subida - 1;
    
    RR = RR[1:] #exclui o primeiro, nao eh um interval RR
    periodo = np.array(RR)
    periodo = np.mean(periodo)
    return periodo

In [ ]:
def detectaRRMod(x,fs):
                
    """ Calculo dos intervalos RR para estimativa do periodo medio do batimento
    por meio do modulo wfdb.

    Args:
        x: numpy.ndarray. Sinal de ECG a partir do qual se deseja estimar os intervalos RR.
        fs: int. Frequencia de amostragem do sinal.

    Return:
        periodo: numpy.float64. Estimativa do periodo de um batimento do sinal x
            em numero de amostras. 

    """
    
    # Deteccao dos complexos QRS por meio do modulo wfdb 
    xqrs = processing.XQRS(x, fs = fs);
    xqrs.detect();
    #wfdb.plot_items(signal=x, ann_samp=[xqrs.qrs_inds]);
    qrs_i = xqrs.qrs_inds;
    rr = np.diff(qrs_i);

    periodo2 = np.array(rr)
    periodo2 = np.mean(periodo2)
    return periodo2

In [ ]:
def separa(x, n, periodo):
                    
    """ Separacao do sinal de ECG por plots em segmentos com duracao igual ao numero de amostras
    de um periodo de um batimento para comparacao entre o modulo wfdb e o algoritmo sem limiares 
    dinamicos desenvolvido. Esta funcao desconsidera a centralizacao de um batimento em torno do QRS, 
    como tratado pelas funcoes dataset(), nao comparando batimentos, mas sim trechos. 
    Serve apenas como comparacao rapida com o modulo wfdb.

    Args:
        x: numpy.ndarray. Sinal de ECG a ser segmentado em trechos.
        n: int. Numero de amostras do sinal que se deseja visualizar.
        periodo: numpy.float64. Estimativa do periodo de um batimento do sinal x
            em numero de amostras, encontrada pelo detectaRR() ou pelo detectaRRMod(). 
        
    Return:
        trechos: list. Lista de numpy arrays contendo os trechos do sinal com duracao igual ao periodo,
        nao tendo o complexo QRS centralizado. 
        
    Exemplo de uso:
        fs = 360
        pasta = 'mit-bih-arrhythmia-database-1.0.0/'
        arqnum = ['100', '101', '102', '103', '104', '105', '106', '107','108', '109', '111', '112', '113', '114', '115', '116',
        '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207','208', '209', '210', 
        '212','213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
        r, sym, annind = importaArquivo(pasta,arqnum[5])
        r = r.p_signal[:,0]
        n = 6000
        periodo = detectaRR(r, fs);
        b1 = separa(r, n, periodo)

        periodo2 = detectaRRMod(r, fs);
        b2 = separa(r, n, periodo2)

        print('periodo sem PanTomp:', periodo, 'periodo com PanTomp:', periodo2)
        
    """
    
    nbatimentos = math.ceil(n/periodo)
    index = np.array(range(int(periodo), n, int(periodo)))
    trechos = np.split(x, index)

    count = 1
    for i in range(nbatimentos):
        plt.subplot(4,4,count)
        plt.plot(trechos[i])
        count += 1
        if (count > 16):
            break
    return trechos

In [ ]:
def importaArquivoChazal(pasta, num):
                        
    """ Importacao das gravacoes, das anotacoes e dos indices em que essas anotacoes ocorrem
    para um paciente. Usada para compor os conjuntos de teste e de treinamento.
    
    Args:
        pasta: string. Nome da pasta em que os arquivos do mit-bih-arrhythmia-database estao localizados.
        num: string. Numero do paciente cujos dados quer-se importar.

    Return:
        record: wfdb.io.record.Record. Objeto do modulo wfdb, contem as duas derivacoes do sinal de ECG
            de cada paciente. Cada derivacao pode ser obtida como no exemplo de uso a seguir.
        sym: list. Lista contendo os simbolos das anotacoes dos batimentos, na abordagem de classificacao
            por batimentos. Caso se deseje usar a abordagem por trechos, usar o atributo de anotacoes aux_note.
            (ann = wfdb.rdann(pasta+'215', 'atr')
            ann.aux_note)
        annind: numpy.ndarray. Contem as posicoes (indices) das anotacoes, e, consequentemente, das
            localizacoes do complexo QRS.
        
    Exemplo de uso:
        fs = 360
        pasta = 'mit-bih-arrhythmia-database-1.0.0/'
        arqnum = ['100', '101', '102', '103', '104', '105', '106', '107','108', '109', '111', '112', '113', '114', '115', '116',
        '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207','208', '209', '210', 
        '212','213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
        r, sym, annind = importaArquivo(pasta,arqnum[5])
        r = r.p_signal[:,0] #corresponde a primeira derivacao
        r = r.p_signal[:,1] #corresponde a segunda derivacao
        
    """
    arq = pasta + num
    record = wfdb.rdrecord(arq)
    ann = wfdb.rdann(arq, 'atr')
    sym = ann.symbol 
    annind = ann.sample
    return record, sym, annind

In [ ]:
def geraSimbolos(pasta, delineation_arq, lead):
                            
    """ Funcao que foi usada para gerar os simbolos com suas respectivas ocorrencias (indices) corrigidas,
    obtidos a partir do algoritmo do MATLAB Ecg-kit. Esse algoritmo em MATLAB eh nao supervisionado, ou seja,
    encontra o complexo QRS, a Onda T e a Onda P, bem como o inicio e o fim de cada elemento,
    sem a necessidade de se possuir anotacoes. Por isso, obtem indices dos complexos QRS que nao correspondem aos
    indices verdadeiros, diferindo em algumas quantidades de amostras, devendo ter o posicionamento dos indices 
    corrigidos para centralizar os complexos QRS antes de entrar nas redes neurais. O uso de um algoritmo nao 
    supervisionadoceh essencial para a etapa de segmentacao ja que o banco de dados do mit-bih nao tem anotacoes 
    de cada elemento do ECG, apenas do complexo QRS. Assim, apos encontrar as delimitacoes de cada elemento no 
    MATLAB, compara-se o complexo QRS em torno do qual o batimento tem seus demais elementos encontrados pelo MATLAB 
    com o QRS das anotacoes do banco, atribuindo-se as caracteristicas encontradas de cada elemento (inicio e fim) 
    pelo MATLAB para esse determinado complexo QRS que foi encontrado. A tolerancia usada foi de um quarto 
    do periodo do batimento de determinado paciente anterior ao indice da anotacao e de um quarto do periodo
    posterior ao indice da anotacao.
    
    Args:
        pasta: string. Nome da pasta em que os arquivos do mit-bih-arrhythmia-database estao localizados.
        delineation_arq: string. Nome do arquivo em formato .mat que foi gerado. Exemplo: delineation_116.mat.
        lead: int com valor 0 ou 1. Corresponde a derivacao em que foi detectado uma maior quantidade de complexo
            QRS. Em geral, um dos leads auxilia mais do que o outro na deteccao dos complexos QRS para determinado
            paciente durante o processamento dos dados pelo Ecg-kit do MATLAB. Para evitar desperdicar dados
            encontraram-se primeiramente os pacientes cujos dados tinham mais complexos QRS detectados no lead0 
            e quais tinham mais no lead1. 
            lead1_arq = ['112','114','115','118','207','215','220','117','212','231','232','233'] 
            # sao os arquivos cujo algoritmo wavedet detecta mais R no lead 1
        
    Return:
        sym_correc: list. Lista contendo 'None' quando nao foi identificado complexo QRS e contendo o simbolo
            corrigido caso contrario.
        indice: list. Lista contendo 'None' quando nao foi identificado complexo QRS e contendo o indice
            do R-peak caso contrario.
        R: list. Lsita contendo os valores de ocorrencia dos complexos QRS nao corrigidos, isto eh, 
            que sairam diretamente do MATLAB.
        
    """
    
    QRSon, QRSoff, T, P, R, nada = waveDelineation(pasta, delineation_arq, lead, simbolos) 
    #nada sao as anotacoes que o algoritmo do matlab preve, que nao sao sempre verdadeiros
    # e nao possuem relevancia para o projeto e estao comentadas no waveDelineation()
    # R sao os indices dos complexos QRS detectados por um algoritmo nao supervisionado
    
    s, sym, a = importaArquivoChazal(pasta, delineation_arq)
    # a sao os indices dos complexos QRS que de fato sao corretos pelas anotacoes originais
    
    rr = np.diff(R)
    rr_finitos = rr[isnan(rr) == False]
    med_rr = np.sum(rr_finitos)/(rr_finitos.shape[0])
    
    indice = [None]*R.shape[0]
    sym_correc = [None]*R.shape[0]
    
    for i in range(R.shape[0]):
        for j in range(a.shape[0]):
            if (((R[i] - med_rr//4) < a[j]) and (a[j] < R[i] + med_rr//4)): #tolerancia usada na correcao
                indice[i] = a[j]
                sym_correc[i] = sym[j] 
                #caso esteja dentro da tolerancia, o simbolo correspondente a anotacao eh o simbolo
                #verdadeiro dado pelo banco de dados original
    return sym_correc, indice, R

In [ ]:
def moving_average(x, w):
                        
    """ Faz a media movel de um vetor. Usado para fazer a media de 10 intervalos RR consecutivos.

    Args:
        x: numpy.ndarray. Sinal cuja media de determinado segmento quer-se calcular.
        w: int. Tamanho da janela em que a media eh calculada, por exemplo, 10 para 10 intervalos RR. 
        
    Return:
        np.hstack((mov, extend_fim)): numpy.ndarray. Contem a media de 10 batimentos consecutivos. Nos
            extremos utiliza 10 posteriores ou anteriores. Quando possivel, utiliza 5 intervalos RR anteriores
            e 5 posteriores.
  
    """
    
    x = np.array((x))
    mov = np.convolve(x, np.ones(w), 'valid') / w
    dif = (x.shape[0] - mov.shape[0])//2
    extend_ini = np.full((dif), mov[0])
    mov = np.hstack((extend_ini, mov))
    extend_fim = np.full((x.shape[0]-np.size(mov)), mov[np.size(mov)-1])
    return np.hstack((mov, extend_fim))
    
def pontos(sig, QRSon, QRSoff, T, R, pos_vec):
                        
    """ Faz a tomada dos 19 pontos de Chazal. Usa o comeco e o fim encontrados para cada elemento do ECG por meio
    do MATLAB para tomar, por meio da interpolacao dos dados (que sao discretos), 10 e 9 amostras do sinal
    uniformemente espacadas, em duas janelas distintas. Funcao chamada em dataset() para cada posicao pos_vec
    correspondente a cada batimento.

    Args:
        sig: numpy.ndarray. Sinal original.
        QRSon: indice (em amostras) do inicio do QRS para determinado batimento (pos_vec).
        QRSoff: indice (em amostras) do fim do QRS para determinado batimento (pos_vec).
        T: indice (em amostras) do fim da Onda T para determinado batimento (pos_vec).
        R: indice (em amostras) de R-peak para determinado batimento (pos_vec).
        pos_vec: int. Batimento especifico em que os elementos estao sendo analisados.
        
    Return:
        f_qrs_int: 10 amostras tomadas uniformemente entre o inicio e o final do QRS.
        xnovo_qrs: localizacao das 10 amostras tomadas em f_qrs_int.
        f_t_int: 9 amostras tomadas uniformemente entre o fim do QRS e o fim da Onda T.
        xnovo_t: localizacao das 9 amostras tomadas em f_t_int.
  
    """
    
    #QRSon-QRSoff
    x_qrs = np.arange(QRSon[pos_vec],QRSoff[pos_vec])
    y_qrs = sig[QRSon[pos_vec].astype(int):QRSoff[pos_vec].astype(int)]
    f_qrs = interp1d(x_qrs, y_qrs)
    xnovo_qrs = np.arange(QRSon[pos_vec], QRSoff[pos_vec], (QRSoff[pos_vec]-QRSon[pos_vec])/10)
    f_qrs_int = f_qrs(xnovo_qrs)
    
    #QRSoff-T
    x_t = np.arange(QRSoff[pos_vec],T[pos_vec])
    y_t = sig[QRSoff[pos_vec].astype(int):T[pos_vec].astype(int)]
    f_t = interp1d(x_t, y_t)
    xnovo_t = np.arange(QRSoff[pos_vec], T[pos_vec], (T[pos_vec]-QRSoff[pos_vec])/9)
    f_t_int = f_t(xnovo_t)

    return f_qrs_int, xnovo_qrs, f_t_int, xnovo_t

def intervaloRR(R):
                        
    """ Reune as quatro informacoes dos intervalos RR de Chazal, que incluem os intervalos RR anteriores a um
    batimento, posteriores a um batimento, o intervalo RR medio de 10 intervalos locais e o intervalo medio
    para um paciente.

    Args:
        R: indices (em amostras) dos R-peak de todos os batimentos de um paciente.
        
    Return:
        pre_rr: intervalos RR anteriores a um batimento.
        pos_rr: intervalos RR posteriores a um batimento.
        local_rr: intervalos RR medios de 10 intervalos locais.
        med_rr: intervalo RR medio de uma gravacao inteira de um paciente.
  
    """
    
    rr = np.abs(np.diff(R))
    rr_finitos = rr[isnan(rr) == False]
    med_rr = np.sum(rr_finitos)/(rr_finitos.shape[0])

    rr[isnan(rr)] = med_rr
    pre_rr = np.hstack(([0], rr))
    pos_rr = np.hstack((rr, [0]))

    local_rr = moving_average(pre_rr, 10)

    return pre_rr, pos_rr, med_rr, local_rr

In [4]:
def waveDelineation(pasta, delineation_arq, lead, simbolos):
                            
    """ Importa do arquivo em .mat os valores das delimitacoes (obtidos da segmentacao) de cada elemento do ECG.

    Args:
        pasta: string. Nome da pasta em que os arquivos do mit-bih-arrhythmia-database estao localizados.
        delineation_arq: string. Nome do arquivo em formato .mat que foi gerado. Exemplo: 'delineation_116.mat'.
        lead: int com valor 0 ou 1. Corresponde a derivacao em que foi detectado uma maior quantidade de complexo
            QRS. Em geral, um dos leads auxilia mais do que o outro na deteccao dos complexos QRS para determinados
            pacientes durante o processamento dos dados pelo Ecg-kit do MATLAB. Para evitar desperdicar dados
            encontraram-se primeiramente os pacientes cujos dados tinham mais complexos QRS detectados no lead0 
            e quais tinham mais no lead1. 
        simbolos: arquivo de json.load(json_file) chamado no notebook das redes neurais, juntamente com a divisao de
            pacientes e de dados. Contem os simbolos corrigidos por meio da funcao  geraSimbolos(pasta, delineation_arq, lead)
            guardados em 'simbolos_versaofinal.json'. Apos rodar uma vez a funcao geraSimbolos(pasta, delineation_arq, lead),
            os simbolos guardados em .json foram usados para todo o projeto, devido ao custo computacional da funcao.
        
    Return:
        QRSon: indices (em amostras) dos inicios dos QRS para todos batimentos de determinado paciente obtidos do ecg-kit.
        QRSoff: indices (em amostras) dos finais dos QRS para todos batimentos de determinado paciente obtidos do ecg-kit.
        T: indices (em amostras) dos finais das Ondas T para todos batimentos de determinado paciente obtidos do ecg-kit.
        P: presenca ou nao (1 ou 0) das Ondas P para todos batimentos de determinado paciente obtidos do ecg-kit.
        R: indices (em amostras) dos R-peaks para todos batimentos de determinado paciente obtidos do ecg-kit.
        sym: simbolos com posicoes corrigidas pelo geraSimbolos(pasta, delineation_arq, lead) para todos 
            batimentos de determinado paciente.
        
    """

    sym = simbolos[delineation_arq]

    lead = str(lead)
    delineation_arq = str(pasta + 'delineation_' + delineation_arq) 
    morph = ['QRSon_lead', 'QRSoff_lead','Toff_lead','P_lead', 'R_lead', 'ann']
    # morph: lista com todas as strings de variaveis desejadas e calculadas no matlab
    delin = sio.loadmat(delineation_arq)

    QRSon = delin[str(morph[0] + lead)]
    QRSon = QRSon.reshape(QRSon.shape[1])
    
    QRSoff = delin[str(morph[1] + lead)]
    QRSoff = QRSoff.reshape(QRSoff.shape[1])

    T = delin[str(morph[2] + lead)]
    T = T.reshape(T.shape[1])

    P = delin[str(morph[3] + lead)]
    P = P.reshape(P.shape[1])

    R = delin[str(morph[4] + lead)]
    R = R.reshape(R.shape[1])

    # ann = delin[str(morph[5])]
    # ann = ann.reshape(ann.shape[0])

    # QRSon = QRSon[QRSon < e].astype(int)
    # QRSoff = QRSoff[QRSoff < e].astype(int)
    # T = T[T < e].astype(int)
    # P = P[P < e].astype(int)
    # R = R[R < e].astype(int)

    return QRSon, QRSoff, T, P, R, sym # ann

In [ ]:
def Transformadas(tipo, sinal, fs, periodo, wavFunc, l):
                                
    """ Funcao que calcula os coeficientes da transformada de Fourier, da transformada de wavelet e de estatisticas
    da transformada de wavelet.

    Args:
        tipo: string. 
            Coloque 'Fourier'caso queira obter os coeficientes da transformada de Fourier para frequencias
            menores do que 60 Hz. Mudar L no caso de outra frequencia. Comentar ou descomentar a normalizacao 
            dos coeficientes (divisao do absoluto pelo maximo do absoluto). Nesse caso, 'wavFunc' e l nao importam.
            Coloque 'Wavelet'caso queira obter os coeficientes da transformada de wavelet, determinados pela mother-wavelet
            wavFunc e pelo nivel de decomposicao l.
            Coloque 'stat_Wav' para obter estatisticas a partir dos coeficientes da transformada de wavelet como feito em 
            "ECG beat classifier designed by combined neural network model" de Inan Guler.
            
        sinal: numpy.array. Sinal ao qual deve ser aplicada a transformada.
        fs: frequencia de amostragem do sinal.
        periodo: periodo do sinal usado para calculo das frequencias da transformada de Fourier.
        wavFunc: string. Mother-wavelet usada na transformada de wavelet. Usar as mesmas strings do modulo pywt.wavedec().
            Por exemplo: 'db2' para daubechies 2
        l: int. Nivel de decomposicao da transformada de wavelet.
            Por exemplo: l=4 gera os coeficientes  A4 D4 D3 D2 D1
        
    Return:
        coefs: numpy.array. Podem ser os coeficientes da transformada de Fourier, da transformada de wavelet, ou
            estatisticas obtidas da transformada de wavelet.
        
    """

    if tipo == 'Fourier':
        freqs = np.fft.fftfreq(periodo, d = 1/fs)
        freqs = freqs[0:periodo//2]
        L = np.size(freqs[freqs < 60])
        coefs = np.fft.fft(sinal)
        coefs = np.concatenate((np.array([0]), coefs[1:L]), axis = 0)
        coefs = np.abs(coefs)/np.max(np.abs(coefs))
    elif tipo == 'Wavelet':
        coefs = np.concatenate([x for x in pywt.wavedec(sinal, wavFunc, level=l)], axis=0)
#         coefs = np.concatenate(pywt.wavedec(sinal, wavFunc, level=l), axis = 0)
#         coefs = coefs/np.max(np.abs(coefs))
    elif tipo == 'stat_Wav':
        win_size = 9
        coefs_wav = pywt.wavedec(sinal, wavFunc, level=l)
        med_abs = [np.mean(y) for y in [abs(x) for x in coefs_wav]]
        med = [abs(x) for x in [np.mean(y) for y in coefs_wav]]
        Pxx = []
        f = []
        avg_pow = [np.sum(x)/x.shape[0] for x in [y**2 for y in coefs_wav]]
        std_dev = [np.std(x) for x in coefs_wav]
        #a ordem eh sempre A4 D4 D3 D2 D1
        #fara na ordem med[4]/med[3] = D1/D2
        #o flip final volta para a ordem A4 D4 D3 D2 D1
        ratio = np.flip([med[i+1]/(med[i]) for i in np.flip(np.arange(size(med)-1))])
        coefs = np.concatenate([med_abs, avg_pow, std_dev, ratio], axis=0)
    return coefs

In [ ]:
def dataset(pasta, arq, periodo, fs, simbolos, modo, norm, wavFunc, level, classes, blocosLSTM):    
                                    
    """ Funcao que realiza essencialmente o papel de dividir os dados entre os conjuntos de teste e de treinamento,
    segmentando os batimentos de modo que os complexos QRS estejam centralizados na entrada da rede no caso do uso
    do sinal de ECG original como entrada.

    Args:
        pasta: string. Nome da pasta em que os arquivos do mit-bih-arrhythmia-database estao localizados.
        arq: lista. Lista contendo strings dos numeros dos pacientes do conjunto usado (DS1 ou DS2).
        periodo: int. Periodo medio de um batimento em amostras. Fixado em 320 para todos os propositos do projeto.
        fs: int. Frequencia de amostragem do sinal.
        simbolos: arquivo de json.load(json_file) chamado no notebook das redes neurais, juntamente com a divisao de
            pacientes e de dados. Contem os simbolos corrigidos por meio da funcao  geraSimbolos(pasta, delineation_arq, lead)
            guardados em 'simbolos_versaofinal.json'. Apos rodar uma vez a funcao geraSimbolos(pasta, delineation_arq, lead),
            os simbolos guardados em .json foram usados para todo o projeto, devido ao custo computacional da funcao.
        modo: string. 
            'bat': um batimento do sinal original.
            '3bat': tres batimentos do sinal original.
            'DFT': coeficientes da transformada de Fourier para freq menores que 60 Hz.
            'chazal': 26 caracteristicas extraidas de Chazal para cada derivacao.
            'interp': 19 das 26 caracteristicas de Chazal, que compoe a tomada dos 19 pontos.
            'caract': 7 das 26 caracteristicas de Chazal, que compoe a duracao do QRS, as informacoes dos intervalos RR e 
                presenca da Onda P.
            'DWT': coeficientes da transformada discreta de wavelet.
            'stat_DWT': estatisticas a partir dos coeficientes da transformada de wavelet como feito em 
                "ECG beat classifier designed by combined neural network model" de Inan Guler.
            'MI': tres batimentos do sinal original e caracteristicas extraidas de Chazal, retornados em 
                vetores distintos, para serem usados em diferentes classificadores e permitir a combinacao deles
                ou ainda, para serem usados em uma mesma CNN, concatenando-os antes de camadas totalmente conectadas.
            'NNA': abordagem em que os batimentos a serem classificados na LSTM nao sao os do passo de iteracao intermediario,
                mas sim os do ultimo tempo de iteracao.
        norm: booleano. (True or False). Indica se o sinal de ECG deve ser normalizado dividindo-se o abs pelo maximo do abs,
            antes da extracao de caracteristicas ou do seu uso como entrada.
        wavFunc: string. Mother-wavelet usada na transformada de wavelet. Usar as mesmas strings do modulo pywt.wavedec().
            Por exemplo: 'db2' para daubechies 2
        level: int. Nivel de decomposicao da transformada de wavelet.
            Por exemplo: l=4 gera os coeficientes  A4 D4 D3 D2 D1
        classes: string. Indica o tipo de problema a ser tratado.
            'NSVF': problema de 4 classes.
            'NSV': problema de 3 classes.
            'NSVFQ': problema de 5 classes.
        blocosLSTM: int. Numero de passos de iteracao para a rede recorrente com LSTM. Tem importancia apenas no
            uso de RNNs com a abordagem em que o ultimo batimento eh o que se deseja classificar ('NNA'),
            podendo ser deixado 0 em qualquer outro caso.
            
    Return:
         X0: exemplos da primeira derivacao. Para cada modo, consiste em uma representacao de diferentes informacoes.
             Pode ser um batimento do sinal original, tres batimentos, coeficientes de transformadas, etc.
         X1: exemplos da segunda derivacao. Para cada modo, consiste em uma representacao de diferentes informacoes.
             Pode ser um batimento do sinal original, tres batimentos, coeficientes de transformadas, etc.
         Y: vetor contendo os valores das classes de cada batimento.
         Sym: simbolos de cada batimento.
         Patient: numero dos pacientes aos quais cada batimento pertence.
         X0_MI e X1_MI: entradas de Chazal. Para obter tanto o sinal original quanto as caracteristicas de Chazal, 
             seja para combinar classificadores ou para concatenar caracteristicas na CNN antes das camadas 
             totalmente conectadas, o modo 'MI' fornece as duas saidas (sinal original + extracao Chazal), 
             com os respectivos valores verdadeiros em Y.
         
    """
    
    e = 1e10
    N = ['N', 'L', 'R', 'e', 'j']
    S = ['A', 'a', 'J', 'S']
    V = ['V', 'E']
    F = ['F']
    Q = ['/', 'f', 'Q']
    ncols = 0
    if classes == 'NSVF':
        all_sym = np.hstack([N, S, V, F])
        df_sym = pd.DataFrame({'Classe': ['N', 'S', 'V', 'F'], 'Sym': [N, S, V, F]})
    elif classes == 'NSV':
        all_sym = np.hstack([N, S, V])
        df_sym = pd.DataFrame({'Classe': ['N', 'S', 'V'], 'Sym': [N, S, V]})
    elif classes == 'NSVFQ':
        all_sym = np.hstack([N, S, V, F, Q])
        df_sym = pd.DataFrame({'Classe': ['N', 'S', 'V', 'F', 'Q'], 'Sym': [N, S, V, F, Q]})
#     janela = 3*periodo #apenas para o DFT e o DWT
    if modo == 'bat':
        janela = periodo
        ncols = periodo
    elif modo == '3bat':
        janela = 3*periodo
        ncols = 3*periodo
    elif modo == 'DFT':
        janela = periodo
        freqs = np.fft.fftfreq(periodo, d = 1/fs)
        freqs = freqs[0:periodo//2]
        L = np.size(freqs[freqs < 60])
        ncols = ncols + L
#     elif modo == 'chazal_1_clf':
#         ncols = 45 #4 (RR), 3 (QRS duration, T duration, P flag), 19 (lead 0), 19 (lead 1)
#     # ncols = 2*janela + 7
    elif modo == 'chazal':
        janela = periodo
        ncols = 26
    elif modo == 'interp':
        janela = periodo
        ncols = 19
    elif modo == 'caract':
        janela = periodo
        ncols = 7
    elif modo == 'DWT':
        janela = periodo
        coefs = pywt.wavedec(np.ones((janela)), wavFunc, level=level, mode='symmetric')
        L = np.size(np.hstack(coefs))
        ncols = ncols + L
    elif modo =='stat_DWT':
        janela = periodo
        ncols = (level+1)*4-1 #[(level + 1) subbands vezes 4 parametros (media, P, dev, razao)] - 1 (pois nao faz ratio do ultimo)
    elif modo == 'MI': #multiple inputs
        janela = 3*periodo
        ncols = 3*periodo
        ncols2 = 26
    elif modo == 'NNA': #normal, normal, arritmia
        janela = blocosLSTM*periodo
        esquerda = periodo*(blocosLSTM*2-1)//2
        direita = periodo//2
        ncols = janela
    X0 = np.zeros((1, ncols)) #.astype(np.complex64)
    X1 = np.zeros((1, ncols)) #.astype(np.complex64)
    if modo == 'MI':
        X0_MI = np.zeros((1, ncols2))
        X1_MI = np.zeros((1, ncols2))
    Y = np.zeros((1,1))

    Sym = []
    Patient = []
    max_rows = []

    lead1_arq = ['112','114','115','118','207','215','220','117','212','231','232','233'] #arquivos cujo algoritmo wavedet detecta mais R no lead 1
    
    for i in arq:
        if isin(i, lead1_arq):
            QRSon, QRSoff, T, P, annind, sym = waveDelineation(pasta, i, 1, simbolos)
        else:
            QRSon, QRSoff, T, P, annind, sym = waveDelineation(pasta, i, 0, simbolos)
        ind_real = np.load(str('indices/' + i + '.npy'), allow_pickle=True)
        sym = np.array((sym))
        pre_rr, pos_rr, med_rr, local_rr = intervaloRR(annind)
        pos_vec = (np.arange(0, np.size(annind)))
        s, symbols, a = importaArquivoChazal(pasta, i)
        s0 = s.p_signal[:,0]
        s1 = s.p_signal[:,1] 
        nrows =  size(isin(sym, all_sym)) ##########
        x0 = np.zeros((nrows, ncols))
        x1 = np.zeros((nrows, ncols))
        if modo == 'MI': #multiple input
            x0_MI = np.zeros((nrows, ncols2))
            x1_MI = np.zeros((nrows, ncols2))
        y = np.zeros((nrows, 1))
        max_row = 0
        sym[isin(sym, all_sym) == False] = None
        
        for pos in pos_vec:
            if (sym[pos] != None):
                df_sym_aux = pd.DataFrame()
                df_sym_aux["bool"] = df_sym["Sym"].apply(lambda j: 1 if sym[pos] in j else 0)
                label = df_sym_aux[df_sym_aux["bool"] == True].index.tolist()
                label = label[0]

                if (QRSon[pos] < e) and (QRSoff[pos] < e) and (T[pos] < e):
                    QRS_dur_i = QRSoff[pos] - QRSon[pos]
                    T_dur_i = T[pos] - QRSoff[pos]
                    P_i = 1.*(P[pos] < e)
                    f_qrs_int0, xnovo_qrs0, f_t_int0, xnovo_t0 = pontos(s0, QRSon, QRSoff, T, annind, pos)
                    f_qrs_int1, xnovo_qrs1, f_t_int1, xnovo_t1 = pontos(s1, QRSon, QRSoff, T, annind, pos)
                    if norm == True:
                        s0 = s0/np.max(np.abs(s0))
                        s1 = s1/np.max(np.abs(s1))
                        f_qrs_int0, xnovo_qrs0, f_t_int0, xnovo_t0 = pontos(s0, QRSon, QRSoff, T, annind, pos)
                        f_qrs_int1, xnovo_qrs1, f_t_int1, xnovo_t1 = pontos(s1, QRSon, QRSoff, T, annind, pos)
                    if modo == 'NNA':
                        sinal0 = s0[ind_real[pos].astype(int) - esquerda : ind_real[pos].astype(int) + direita]
                        sinal1 = s1[ind_real[pos].astype(int) - esquerda : ind_real[pos].astype(int) + direita]
#                         sinal0 = s0[ind_real[pos].astype(int) - 5*janela//6 : ind_real[pos].astype(int) + janela//6]
#                         sinal1 = s1[ind_real[pos].astype(int) - 5*janela//6 : ind_real[pos].astype(int) + janela//6]
                    else:
                        sinal0 = s0[ind_real[pos].astype(int) - janela//2 : ind_real[pos].astype(int) + janela//2]
                        sinal1 = s1[ind_real[pos].astype(int) - janela//2 : ind_real[pos].astype(int) + janela//2]
                    
                    if np.size(sinal0) == janela:
                        if norm == True:
                            sinal0 = sinal0/np.max(np.abs(sinal0))
                            sinal1 = sinal1/np.max(np.abs(sinal1))
                        if (modo == 'chazal'):
                            x0[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i, f_qrs_int0, f_t_int0))
                            x1[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i, f_qrs_int1, f_t_int1))
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif(modo == 'interp'):
                            x0[max_row, :] = np.hstack((f_qrs_int0, f_t_int0))
                            x1[max_row, :] = np.hstack((f_qrs_int1, f_t_int1))
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif(modo == 'caract'):
                            x0[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i))
                            x1[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i))
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif (modo == 'bat') or (modo == '3bat') or (modo == 'NNA'):
                            x0[max_row, :] = sinal0
                            x1[max_row, :] = sinal1
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif (modo == 'DFT'):
                            x0[max_row, :] = Transformadas('Fourier', sinal0, fs, periodo, wavFunc, level)
                            x1[max_row, :] = Transformadas('Fourier', sinal1, fs, periodo, wavFunc, level)
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif (modo == 'DWT'):
                            x0[max_row, :] = Transformadas('Wavelet', sinal0, fs, periodo, wavFunc, level)
                            x1[max_row, :] = Transformadas('Wavelet', sinal1, fs, periodo, wavFunc, level)
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif (modo == 'stat_DWT'):
                            x0[max_row, :] = Transformadas('stat_Wav', sinal0, fs, periodo, wavFunc, level)
                            x1[max_row, :] = Transformadas('stat_Wav', sinal1, fs, periodo, wavFunc, level)
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
                        elif (modo == 'MI'):
                            x0[max_row, :] = sinal0
                            x1[max_row, :] = sinal1
                            x0_MI[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i, f_qrs_int0, f_t_int0))
                            x1_MI[max_row, :] = np.hstack((pre_rr[pos], pos_rr[pos], med_rr, local_rr[pos], QRS_dur_i, T_dur_i, P_i, f_qrs_int1, f_t_int1))
                            y[max_row:] = label
                            Sym.append(sym[pos])
                            Patient.append(i)
                            max_row += 1
        x0 = x0[:max_row,:]
        x1 = x1[:max_row,:]
        if (modo == 'MI'):
            x0_MI = x0_MI[:max_row,:]
            x1_MI = x1_MI[:max_row,:]
        y = y[:max_row,:]
        max_rows.append(max_row)
        X0 = np.append(X0, x0, axis = 0)
        X1 = np.append(X1, x1, axis = 0)
        if (modo == 'MI'):
            X0_MI = np.append(X0_MI, x0_MI, axis = 0)
            X1_MI = np.append(X1_MI, x1_MI, axis = 0)
        Y = np.append(Y, y, axis = 0)
        
    X0 = X0[1:,:]
    X1 = X1[1:,:]
    if(modo == 'MI'):
        X0_MI = X0_MI[1:,:]
        X1_MI = X1_MI[1:,:]
    Y = Y[1:,:]
    
    if(modo == 'MI'):
        return X0, X1, X0_MI, X1_MI, Y, Sym, Patient
    return X0, X1, Y, Sym, Patient

In [ ]:
def metricas(M, salvar, arq, modelo, entrada, periodo, param, nepochs, key):
                            
    """ Calculo das metricas seguindo-se as orientacoes da AAMI.

    Args:
        M: matriz de confusao do conjunto de teste.
        salvar: booleano. Colocar True caso queira salvar os dados em .csv, com as metricas de cada classe.
        arq: string. Nome do arquivo em que as metricas devem ser salvas.
        modelo: string. Nome do modelo treinado e testado e que sera usado em uma das colunas, junto as metricas salvas.
        entrada: string. Nome da entrada usada no modelo que sera salva em uma das colunas, junto as metricas.
        periodo: string. Periodo medio de um batimento considerado que sera salvo em uma das colunas, junto as metricas.
        param: string. Hiperparam usados, como valor de dropout, numero de neuronios, numero de camadas,
            que serao salvos em uma coluna, junto as metricas, para lembrar a estrutura que forneceu determinado
            resultado.
        nepochs: string. Numero de epocas usado, que sera salvo junto as metricas.
        key: string. Identificador do modelo para relacionar as metricas obtidas ao sumario salvo para cada modelo
            em um txt.
        
    Return:
        print() das metricas de Sensibilidade e de Precisao para cada classe. Descomentar caso desejado. 
  
    """
    
    TN = M[0,0]
    TPv = M[2,2]
    TPs = M[1,1]
    TPf = M[3,3]
    TPq = M[4,4]
    
    Sensi_F = 100*TPf/np.sum(M[3,:])
    Sensi_Q = 100*TPq/np.sum(M[4,:])
    
    Sensi_N = 100*TN/np.sum(M[0,:])
    Preci_N = 100*TN/(TN + np.sum(M[1:,0]))
    #print('------Normal------')
    #print('\n', 'Sensibilidade:', Sensi_N, '\n','Precisão:', Preci_N, '\n')

    TNs = M[0,0] + np.sum(M[0, 2:]) + np.sum(M[2:, 0]) + np.sum(M[2:, 2:])
    FNs = np.sum(M[1, 0]) + np.sum(M[1, 2:])
    TPs = M[1,1]
    FPs = M[0, 1] + np.sum(M[2:4, 1])
    Sensi_SVEB = 100*TPs/(TPs + FNs)
    Spec_SVEB = 100*TNs/(TNs + FPs)
    Preci_SVEB = 100*TPs/(TPs + FPs)
    FPR_SVEB = 100*FPs/(TNs + FPs)
    Acc_SVEB = 100*(TPs + TNs)/(TPs + TNs + FPs + FNs)
    HM_SVEB = 2*Sensi_SVEB*Spec_SVEB/(Sensi_SVEB + Spec_SVEB)
    #print('------SVEB------')
    #print('\n', 'Sensibilidade:', Sensi_SVEB, '\n','Precisão:', Preci_SVEB, '\n', 'Taxa de Falsos Positivos:', FPR_SVEB, '\n', 'Acurácia:', Acc_SVEB, '\n', 'HM:', HM_SVEB, '\n')

    TNv = np.sum(M[0:2, 0:2]) + np.sum(M[3:, 0:2]) + np.sum(M[0:2, 3:]) + np.sum(M[3:, 3:])
    FNv = np.sum(M[2, 0:2]) + np.sum(M[2, 3:])
    TPv = M[2,2]
    FPv = M[0, 2] + M[1, 2]
    Sensi_VEB = 100*TPv/(TPv + FNv)
    Spec_VEB = 100*TNv/(TNv + FPv)
    Preci_VEB = 100*TPv/(TPv + FPv)
    FPR_VEB = 100*FPv/(TNv + FPv)
    Acc_VEB = 100*(TPv + TNv)/(TPv + TNv + FPv + FNv)
    HM_VEB = 2*Sensi_VEB*Spec_VEB/(Sensi_VEB + Spec_VEB)
    #print('------VEB------')
    #print('\n', 'Sensibilidade:', Sensi_VEB, '\n', 'Precisão:', Preci_VEB, '\n', 'Taxa de Falsos Positivos:', FPR_VEB, '\n', 'Acurácia:', Acc_VEB, '\n', 'HM:', HM_VEB, '\n')

    #print('------F------')
    FPf = np.sum(M[0:3,3]) + np.sum(M[4:,3])
    Preci_F = 100*TPf/(TPf + FPf)
    #print('\n', 'Sensibilidade:', Sensi_F, '\n', 'Precisão:', Preci_F , '\n')

    #print('------Q------')
    FPq = np.sum(M[0:4])
    Preci_Q = 100*TPq/(TPq + FPq)
    #print('\n', 'Sensibilidade:', Sensi_Q, '\n', 'Precisão:', Preci_Q , '\n')
    
    Acc_total = 100*(TN+TPs+TPv+TPf+TPq)/np.sum(np.sum(M, axis = 1),axis = 0)
    #print('\n','\n','Acurácia total:', Acc_total)
    
    if salvar == True:
        fields=[str(x) for x in [modelo, entrada, periodo, param, nepochs, key, Acc_total, Sensi_N, Preci_N, Sensi_SVEB, Preci_SVEB, Sensi_VEB, Preci_VEB,Sensi_F, Preci_F, Sensi_Q, Preci_Q]]
        with open(arq, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)

In [ ]:
def metricasNSVF(M, salvar, arq, modelo, entrada, periodo, param, nepochs, key):
                                
    """ Equivalente da funcao metricas() para o problema de 4 classes. 
    Calculo das metricas seguindo-se as orientacoes da AAMI.

    Args:
        M: matriz de confusao do conjunto de teste.
        salvar: booleano. Colocar True caso queira salvar os dados em .csv, com as metricas de cada classe.
        arq: string. Nome do arquivo em que as metricas devem ser salvas.
        modelo: string. Nome do modelo treinado e testado e que sera usado em uma das colunas, junto as metricas salvas.
        entrada: string. Nome da entrada usada no modelo que sera salva em uma das colunas, junto as metricas.
        periodo: string. Periodo medio de um batimento considerado que sera salvo em uma das colunas, junto as metricas.
        param: string. Hiperparam usados, como valor de dropout, numero de neuronios, numero de camadas,
            que serao salvos em uma coluna, junto as metricas, para lembrar a estrutura que forneceu determinado
            resultado.
        nepochs: string. Numero de epocas usado, que sera salvo junto as metricas.
        key: string. Identificador do modelo para relacionar as metricas obtidas ao sumario salvo para cada modelo
            em um txt.
        
    Return:
        print() das metricas de Sensibilidade e de Precisao para cada classe. Descomentar caso desejado. 
  
    """
    
    TN = M[0,0]
    TPv = M[2,2]
    TPs = M[1,1]
    TPf = M[3,3]
    Spec = 100*TN/np.sum(M[0,:])
    Sensi_F = 100*TPf/np.sum(M[3,:])
    
    Sensi_N = 100*TN/np.sum(M[0,:])
    Preci_N = 100*TN/(TN + np.sum(M[1:,0]))
    #print('------Normal------')
    #print('\n', 'Sensibilidade:', Sensi_N, '\n','Precisão:', Preci_N, '\n')

    TNs = M[0,0] + np.sum(M[0, 2:]) + np.sum(M[2:, 0]) + np.sum(M[2:, 2:])
    FNs = np.sum(M[1, 0]) + np.sum(M[1, 2:])
    TPs = M[1,1]
    FPs = M[0, 1] + np.sum(M[2:4, 1])
    Sensi_SVEB = 100*TPs/(TPs + FNs)
    Spec_SVEB = 100*TNs/(TNs + FPs)
    Preci_SVEB = 100*TPs/(TPs + FPs)
    FPR_SVEB = 100*FPs/(TNs + FPs)
    Acc_SVEB = 100*(TPs + TNs)/(TPs + TNs + FPs + FNs)
    HM_SVEB = 2*Sensi_SVEB*Spec_SVEB/(Sensi_SVEB + Spec_SVEB)
    #print('------SVEB------')
    #print('\n', 'Sensibilidade:', Sensi_SVEB, '\n','Precisão:', Preci_SVEB, '\n', 'Taxa de Falsos Positivos:', FPR_SVEB, '\n', 'Acurácia:', Acc_SVEB, '\n', 'HM:', HM_SVEB, '\n')

    TNv = np.sum(M[0:2, 0:2]) + np.sum(M[3:, 0:2]) + np.sum(M[0:2, 3:]) + np.sum(M[3:, 3:])
    FNv = np.sum(M[2, 0:2]) + np.sum(M[2, 3:])
    TPv = M[2,2]
    FPv = M[0, 2] + M[1, 2]
    Sensi_VEB = 100*TPv/(TPv + FNv)
    Spec_VEB = 100*TNv/(TNv + FPv)
    Preci_VEB = 100*TPv/(TPv + FPv)
    FPR_VEB = 100*FPv/(TNv + FPv)
    Acc_VEB = 100*(TPv + TNv)/(TPv + TNv + FPv + FNv)
    HM_VEB = 2*Sensi_VEB*Spec_VEB/(Sensi_VEB + Spec_VEB)
    #print('------VEB------')
    #print('\n', 'Sensibilidade:', Sensi_VEB, '\n', 'Precisão:', Preci_VEB, '\n', 'Taxa de Falsos Positivos:', FPR_VEB, '\n', 'Acurácia:', Acc_VEB, '\n', 'HM:', HM_VEB, '\n')

    #print('------F------')
    FPf = np.sum(M[0:3,3]) + np.sum(M[4:,3])
    Preci_F = 100*TPf/(TPf + FPf)
    #print('\n', 'Sensibilidade:', Sensi_F, '\n', 'Precisão:', Preci_F , '\n')
    
    Acc_total =  100*(TN+TPs+TPv+TPf)/np.sum(np.sum(M, axis = 1),axis = 0)
    #print('\n','\n','Acurácia total:', Acc_total)
    
    if salvar == True:
        fields=[str(x) for x in [modelo, entrada, periodo, param, nepochs, key, Acc_total, Sensi_N, Preci_N, Sensi_SVEB, Preci_SVEB, Sensi_VEB, Preci_VEB,Sensi_F, Preci_F, '-', '-']]
        with open(arq, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)

In [ ]:
#@title
def plotECG(morf, X_train, id1, id2, id3, id4, id5):
                                    
    """ Funcao para plotar os sinais de ECG para todas as classes.

    Args:
        morf: booleano. True se caracteristicas de Chazal, False se sinal original. 
        X_train: numpy.array. Dados usados para plot, sejam de treinamento ou de teste.
        id1, id2, id3, id4 e id5: indices de ocorrencia das classes N, S, V, F e Q, respectivamente, que se
            desejam plotar.
  
    """
    
    if(morf == True):
        carac = ['Pré-RR', 'Pós-RR', 'Med-RR Total', 'Med-RR local', 'duração do QRS', 'duração da onda T', 'presença da onda P']
        plt.figure(figsize=[15,5])
        plt.xticks(np.arange(0,7,1), carac)
        plt.plot(carac, X_train[id1, 0:7], '-ob')
        plt.plot(carac, X_train[id2, 0:7], '-or')
        plt.plot(carac, X_train[id3, 0:7], '-og')
        plt.plot(carac, X_train[id4, 0:7], '-ok')
        plt.plot(carac, X_train[id5, 0:7], '-om')
        plt.title('Características')
        plt.legend(['N', 'S', 'V', 'F', 'Q'])
        plt.ylabel('Amostras')
        plt.show()
        plt.plot(X_train[id1, 7:], '-ob')
        plt.title('Interpolação de ECG da classe N')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id2, 7:], '-or')
        plt.title('Interpolação de ECG da classe S')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id3, 7:], '-og')
        plt.title('Interpolação de ECG da classe V')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id4, 7:], '-ok')
        plt.title('Interpolação de ECG da classe F')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id5, 7:], '-om')
        plt.title('Interpolação de ECG da classe Q')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
    else:
        plt.plot(X_train[id1,:], 'b')
        plt.title('ECG da classe N')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id2,:], 'r')
        plt.title('ECG da classe S')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id3,:], 'g')
        plt.title('ECG da classe V')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id4,:], 'k')
        plt.title('ECG da classe F')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id5,:], 'm')
        plt.title('ECG da classe Q')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()

In [ ]:
def plotECG_NSV(morf, X_train, id1, id2, id3):
                                        
    """ Equivalente da funcao plotECG() para o problema de 3 classes.
    Funcao para plotar os sinais de ECG para as classes N, S e V.

    Args:
        morf: booleano. True se caracteristicas de Chazal, False se sinal original. 
        X_train: numpy.array. Dados usados para plot, sejam de treinamento ou de teste.
        id1, id2 e id3: indices de ocorrencia das classes N, S e V, respectivamente, que se
            desejam plotar.
  
    """
  
    if(morf == True):
        carac = ['Pré-RR', 'Pós-RR', 'Med-RR Total', 'Med-RR local', 'duração do QRS', 'duração da onda T', 'presença da onda P']
        plt.figure(figsize=[15,5])
        plt.xticks(np.arange(0,7,1), carac)
        plt.plot(carac, X_train[id1, 0:7], '-ob')
        plt.plot(carac, X_train[id2, 0:7], '-or')
        plt.plot(carac, X_train[id3, 0:7], '-og')
        plt.title('Características')
        plt.legend(['N', 'S', 'V'])
        plt.ylabel('Amostras')
        plt.show()
        plt.plot(X_train[id1, 7:], '-ob')
        plt.title('Interpolação de ECG da classe N')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id2, 7:], '-or')
        plt.title('Interpolação de ECG da classe S')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id3, 7:], '-og')
        plt.title('Interpolação de ECG da classe V')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
    else:
        plt.plot(X_train[id1,:], 'b')
        plt.title('ECG da classe N')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id2,:], 'r')
        plt.title('ECG da classe S')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()
        plt.plot(X_train[id3,:], 'g')
        plt.title('ECG da classe V')
        plt.ylabel('Amplitude (mV)')
        plt.xlabel('Amostras')
        plt.show()

In [ ]:
def combiner(cond0, cond1):
                
    """ Funcao para combinar classificadores por meio da multiplicacao elemento a elemento.

    Args:
        cond0: Probabilidade condicional de dado exemplo pertencer a certa classe, colocada como posicao de um vetor
            de saida do classificador 0.
        cond1: Probabilidade condicional de dado exemplo pertencer a certa classe, colocada como posicao de um vetor
            de saida do classificador 1.
    Return:
        np.argmax(np.divide(num, denom), axis=1): posicao (classe) do maior valor obtido da operacao 
            de multiplicacao elemento a elemento.

    """
    
    num = np.multiply(cond0, cond1)
    denom = np.sum(num, axis=1).reshape(cond0.shape[0], -1)
    return np.argmax(np.divide(num, denom), axis=1)

def combiner2(cond0, cond1):
                    
    """ Forma alternativa da funcao combiner() para retornar todo o vetor resultante da operacao de combinacao, nao apenas 
    a posicao da classe mais provavel. Permite combinar mais de dois classificadores em cascata.
    
    Args:
        cond0: Probabilidade condicional de dado exemplo pertencer a certa classe, colocada como posicao de um vetor
            de saida do classificador 0.
        cond1: Probabilidade condicional de dado exemplo pertencer a certa classe, colocada como posicao de um vetor
            de saida do classificador 1.
    Return:
        np.divide(num, denom): vetor contendo o valor de saida da operacao de multiplicacao elemento a elemento, para
        cada posicao (classe).

    """
    
    num = np.multiply(cond0, cond1)
    denom = np.sum(num, axis=1).reshape(cond0.shape[0], -1)
    return np.divide(num, denom)

def resultados(Y_real_train, Y_train_preds, Y_real_test, Y_test_preds, numclass):
                        
    """ Funcao para visualizar os resultados tanto para o conjunto de teste quanto para o conjunto de treinamento.
    
    Args:
        Y_real_train: numpy.array. Valores verdadeiros dos exemplos de treinamento.
        Y_train_preds: numpy.array. Valores obtidos pelo classificador para os exemplos de treinamento, apos treinada
            a rede ou apos calculados os parametros do classificador, como na LDA.
        Y_real_test: numpy.array. Valores verdadeiros dos exemplos de teste.
        Y_test_preds: numpy.array. Valores obtidos pelo classificador para os exemplos de teste.
        numclass: int. Numero de classes.
        
    Return:
        M: matriz de confusao do teste.
        print() das metricas (sem considerar a AAMI)

    """
    
    print('-------------------------Treino------------------------')
    print(confusion_matrix(Y_real_train, Y_train_preds))
    print(classification_report(Y_real_train, Y_train_preds, digits = numclass))
    print('-------------------------Teste------------------------')
    M = confusion_matrix(Y_real_test, Y_test_preds)
    print(confusion_matrix(Y_real_test, Y_test_preds))
    print(classification_report(Y_real_test, Y_test_preds, digits = numclass))
    if numclass == 4:
        df_cm = pd.DataFrame(np.divide(M, np.tile(np.sum(M,axis=1), (4,1)).T), index = [i for i in "NSVF"], columns = [i for i in "NSVF"])
    elif numclass == 5:
        df_cm = pd.DataFrame(np.divide(M, np.tile(np.sum(M,axis=1), (5,1)).T), index = [i for i in "NSVFQ"], columns = [i for i in "NSVFQ"])
    else:
        df_cm = pd.DataFrame(np.divide(M, np.tile(np.sum(M,axis=1), (3,1)).T), index = [i for i in "NSV"], columns = [i for i in "NSV"])
    plt.figure(figsize = (10,7))
    ax = sn.heatmap(df_cm, annot=True,cmap="OrRd")
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    return M

def printsum(summ):
                            
    """ Funcao para salvar o summary() das redes.
    
    """
    
    with open(r'summ_rnn.txt','a') as f:
        print(summ, file=f)

def onehot(X, Y, numclass):
                            
    """ Funcao para transformar os valores verdadeiros de inteiros das classes (k = 0, 1... 4) 
    para representacoes one-hot ([1,0,0,0,0], [0,1,0,0,0],..., [0,0,0,0,1]).
    
    Args:
        X: numpy.array. Exemplos.
        Y: numpy.array. Contem os inteiros dos valores verdadeiros dos exemplos.
        numclass: int. Numero de classes.
        
    Return:
        Y: numpy.array. Vetor de dimensao (X.shape[0], numclass), com a representacao 1 entre m dos valores verdadeiros.

    """
    
    Y = np.eye(numclass)[Y.astype('int32')]
    Y = Y.T.reshape(numclass, X.shape[0])
    Y = Y.T
    return Y

def take_ind(ind, X0, X1, Y):
                                
    """ Funcao para obter os dados de apenas determinados indices, permitindo diminuir a quantidade de dados de
    determinadas classes. 
    
    Args:
        ind: numpy.array. Contem os indices que devem ser tomados do conjunto original para o conjunto a ser retornado.
        X0: numpy.array. Conjunto original de exemplos da primeira derivacao.
        X1: numpy.array. Conjunto original de exemplos da segunda derivacao.
        Y: numpy.array. Conjunto original dos valores verdadeiros dos exemplos.
        
    Return:
        X0[ind,:], X1[ind,:], Y[ind]: Conjuntos X0, X1 e Y tomados nos indices ind.

    """
    
    return X0[ind,:], X1[ind,:], Y[ind]

